In [3]:
import numpy as np
import pandas as pd

from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

import networkx as nx
import gurobipy as gp
from gurobipy import GRB

import time

import matplotlib.pyplot as plt
import seaborn as sns

import csv
import os

In [4]:
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering

from sklearn.metrics import (
    normalized_mutual_info_score,
    homogeneity_completeness_v_measure,
    fowlkes_mallows_score,
    adjusted_rand_score
)
from itertools import combinations

In [5]:
import warnings
warnings.filterwarnings('ignore')

## GCS-Q Algorithm

In [ ]:
# Step 1: Construct the initial graph from the adjacency matrix
def construct_graph(adj_matrix):
    G = nx.Graph()
    num_nodes = len(adj_matrix)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if adj_matrix[i][j] != 0:
                G.add_edge(i, j, weight=adj_matrix[i][j])
    return G

# Step 2: Define the function to calculate coalition value (sum of weights)
def coalition_value(subgraph):
    return subgraph.size(weight='weight')


def get_qubo_matrix(W):
    """Computes the QUBO matrix for the Minimum Cut problem given a weight matrix W."""
    n = W.shape[0]  # Number of nodes
    Q = np.zeros((n, n))  # Initialize QUBO matrix
    
    for i in range(n):
        Q[i, i] = np.sum(W[i])  # Diagonal terms (degree of node)
        for j in range(n):
            if i != j:
                Q[i, j] = -W[i, j]  # Off-diagonal terms (negative adjacency)
    
    return Q

# Step 3: Bipartitioning using QUBO and Quantum Annealing
def bipartition(graph):
    if len(graph.nodes())==1:
       return [], [0], 0

    w = nx.adjacency_matrix(graph).todense()
    qubo = get_qubo_matrix(W = w)

    bqm = BinaryQuadraticModel.from_qubo(qubo)

    sampler = EmbeddingComposite(DWaveSampler(token = open('dwave-api-token.txt','r').read(), solver={'topology__type': 'pegasus'}))
    sampleset = sampler.sample(bqm, num_reads=1000)
    qpu_access_time = sampleset.info['timing']['qpu_access_time']

    solution = sampleset.first.sample
    partition1 = [node for node in solution if solution[node] == 1]
    partition2 = [node for node in solution if solution[node] == 0]

    return partition1, partition2, qpu_access_time

def gurobi_qubo_solver_old(linear,quadratic):
    qubo_matrix = np.zeros([len(linear),len(linear)])
    for key,value in linear.items():
        qubo_matrix[int(key),int(key)] = value
    for key,value in quadratic.items():
        qubo_matrix[int(key[0]),int(key[1])] = value/2
        qubo_matrix[int(key[1]),int(key[0])] = value/2
    n = qubo_matrix.shape[0]
    model = gp.Model()
    x = model.addVars(n, vtype=GRB.BINARY)
    obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
    model.setObjective(obj_expr)
    model.setParam('OutputFlag', 0)
    model.optimize()
    if model.status == GRB.OPTIMAL:
        solution = [int(x[i].X) for i in range(n)]
        binary_string = ''.join(str(bit) for bit in solution)
        return binary_string, model.objVal
    else:
        return None, None

def gurobi_qubo_solver(qubo_matrix):
    n = qubo_matrix.shape[0]
    model = gp.Model()
    x = model.addVars(n, vtype=GRB.BINARY)
    obj_expr = gp.quicksum(qubo_matrix[i, j] * x[i] * x[j] for i in range(n) for j in range(n))
    model.setObjective(obj_expr)
    model.setParam('OutputFlag', 0)
    model.optimize()
    if model.status == GRB.OPTIMAL:
        solution = [int(x[i].X) for i in range(n)]
        binary_string = ''.join(str(bit) for bit in solution)
        return binary_string, model.objVal
    else:
        return None, None

# Step 3: Bipartitioning using QUBO and Quantum Annealing
def bipartition_gurobi(graph):
    if len(graph.nodes())==1:
       return [], [0]
    w = nx.adjacency_matrix(graph).todense()
    qubo = get_qubo_matrix(W = w)
    solution_str, objective_value = gurobi_qubo_solver(qubo)
    solution = {idx:int(bit) for idx,bit in enumerate(solution_str)}
    partition1 = [node for node in solution if solution[node] == 1]
    partition2 = [node for node in solution if solution[node] == 0]

    return partition1, partition2


# Step 4: Iterative GCS-Q Algorithm
def gcs_q_algorithm(adj_matrix, qubo_solver = "dwave"):
    G = construct_graph(adj_matrix)
    grand_coalition = list(G.nodes)
    queue = [grand_coalition]
    CS_star = []
    total_qpu_access_time = 0
    total_partition_time = 0

    while queue:
        C = queue.pop(0)  # Dequeue the first coalition
        subgraph = G.subgraph(C).copy()

        # Solve the optimal split problem
        if qubo_solver == "dwave":
            t0 = time.time()
            partition1, partition2, qpu_access_time = bipartition(subgraph)
            partition_time = time.time() - t0
            total_qpu_access_time += qpu_access_time
            total_partition_time += partition_time
        elif qubo_solver == "gurobi":
            partition1, partition2 = bipartition_gurobi(subgraph)
        partition1 = [C[subgraph_node_index] for subgraph_node_index in partition1]
        partition2 = [C[subgraph_node_index] for subgraph_node_index in partition2]

        if not partition2:  # If no meaningful split is found
            CS_star.append(partition1)
        elif not partition1:
            CS_star.append(partition2)
        else:  # If a meaningful split is found, enqueue the partitions
            queue.append(partition1)
            queue.append(partition2)
    

    return CS_star, total_qpu_access_time/10**6, total_partition_time

## PAM Algorithm

In [8]:
def calculate_total_cost(distance_matrix, medoids, clusters):
    total_cost = 0
    for medoid, cluster in zip(medoids, clusters):
        total_cost += np.sum(distance_matrix[cluster][:, medoid])
    return total_cost

def assign_clusters(distance_matrix, medoids):
    clusters = [[] for _ in range(len(medoids))]
    for i in range(distance_matrix.shape[0]):
        distances_to_medoids = [distance_matrix[i, medoid] for medoid in medoids]
        closest_medoid = np.argmin(distances_to_medoids)
        clusters[closest_medoid].append(i)
    return clusters

def pam(distance_matrix, k, max_iter=100):
    # Step 1: Initialize medoids
    medoids = np.random.choice(distance_matrix.shape[0], k, replace=False)
    best_medoids = medoids.copy()
    clusters = assign_clusters(distance_matrix, medoids)
    best_cost = calculate_total_cost(distance_matrix, medoids, clusters)
    
    for _ in range(max_iter):
        for medoid_idx in range(k):
            current_medoid = medoids[medoid_idx]
            non_medoids = [i for i in range(distance_matrix.shape[0]) if i not in medoids]
            
            for new_medoid in non_medoids:
                new_medoids = medoids.copy()
                new_medoids[medoid_idx] = new_medoid
                new_clusters = assign_clusters(distance_matrix, new_medoids)
                new_cost = calculate_total_cost(distance_matrix, new_medoids, new_clusters)
                
                if new_cost < best_cost:
                    best_cost = new_cost
                    best_medoids = new_medoids.copy()
                    clusters = new_clusters
                    
        if np.array_equal(best_medoids, medoids):
            break
        else:
            medoids = best_medoids.copy()
    
    return best_medoids, clusters

def clusters_as_set_of_sets(clusters):
    return [cluster for cluster in clusters]



## K-Means

In [9]:

def run_kmeans(adjacency_matrix, k, seed=None):
    from sklearn.manifold import MDS
    mds = MDS(n_components=2, dissimilarity='euclidean', random_state=seed)
    coords = mds.fit_transform(adjacency_matrix)
    
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=seed)
    labels = kmeans.fit_predict(coords)
    
    clusters = [[] for _ in range(k)]
    for idx, label in enumerate(labels):
        clusters[label].append(idx)
    return clusters

## Agglomerative Hierarchical

In [10]:
def run_hierarchical(adjacency_matrix, k):
    model = AgglomerativeClustering(n_clusters=k)
    labels = model.fit_predict(adjacency_matrix)
    
    clusters = [[] for _ in range(k)]
    for idx, label in enumerate(labels):
        clusters[label].append(idx)
    return clusters


## Spectral Clustering

In [ ]:
def run_spectral(adjcency_matrix, k, seed=None):
    model = SpectralClustering(n_clusters=k, affinity='rbf', random_state=seed)
    labels = model.fit_predict(adjcency_matrix)
    
    clusters = [[] for _ in range(k)]
    for idx, label in enumerate(labels):
        clusters[label].append(idx)
    return clusters

## DIANA (Divisive Hierarchical)

In [75]:
import numpy as np

def diana(dist_matrix, k):
    n = dist_matrix.shape[0]
    clusters = [list(range(n))]

    while len(clusters) < k:
        # Select the cluster with the largest diameter
        diameters = [np.max(dist_matrix[np.ix_(c, c)]) for c in clusters]
        idx_to_split = np.argmax(diameters)
        cluster = clusters.pop(idx_to_split)

        # 1. Find point most dissimilar to all others
        avg_dists = np.mean(dist_matrix[np.ix_([i for i in cluster], [j for j in cluster])], axis=1)
        split_seed_idx = cluster[np.argmax(avg_dists)]
        new_cluster = [split_seed_idx]
        remaining = set(cluster) - {split_seed_idx}

        # 2. Move points one by one to new cluster if it decreases average dissimilarity
        for i in remaining:
            in_old = cluster.copy()
            in_old.remove(i)
            old_avg = np.mean([dist_matrix[i][j] for j in in_old])

            new_avg = np.mean([dist_matrix[i][j] for j in new_cluster])
            if new_avg < old_avg:
                new_cluster.append(i)
            else:
                in_old.append(i)

        old_cluster = list(set(cluster) - set(new_cluster))
        if old_cluster: clusters.append(old_cluster)
        if new_cluster: clusters.append(new_cluster)

    return clusters


## Utility functions

In [76]:
def clusters_to_labels(cluster_list, n_nodes):
    labels = np.zeros(n_nodes, dtype=int)
    for i, cluster in enumerate(cluster_list):
        for node in cluster:
            labels[node] = i
    return labels

In [77]:
def check_and_prepare_csv(report_filename_csv, fieldnames):
    if os.path.exists(report_filename_csv):
        with open(report_filename_csv, mode='r', newline='') as report_file:
            reader = csv.reader(report_file)
            existing_header = next(reader, None)

        if existing_header == fieldnames:
            print("✅ File exists. Headers match. Will start appending.")
        else:
            raise ValueError(
                f"❌ File '{report_filename_csv}' exists but headers do not match.\n"
                f"Expected: {fieldnames}\nFound:    {existing_header}"
            )
    else:
        with open(report_filename_csv, mode='w', newline='') as report_file:
            writer = csv.DictWriter(report_file, fieldnames=fieldnames)
            writer.writeheader()
        print("✅ File created and header written.")

## Generate Synthetic Data

In [ ]:
def entropy_based_cluster_sizes(n_clusters, total_nodes, entropy, seed=None):
    if seed is not None:
        np.random.seed(seed)

    cluster_sizes = np.ones(n_clusters, dtype=int)  # Each cluster gets at least one node
    remaining = total_nodes - n_clusters

    if entropy == 1.0:
        # Uniform distribution
        base = remaining // n_clusters
        extra = remaining % n_clusters
        cluster_sizes += base
        for i in range(extra):
            cluster_sizes[i] += 1
    elif entropy == 0.0:
        # All remaining nodes go to one randomly chosen cluster
        idx = np.random.randint(n_clusters)
        cluster_sizes[idx] += remaining
    else:
        # Mix between entropy=0 and entropy=1
        max_alloc = int((1 - entropy) * remaining)
        min_alloc = remaining - max_alloc

        # Step 1: allocate max_alloc to one cluster
        idx = np.random.randint(n_clusters)
        cluster_sizes[idx] += max_alloc

        # Step 2: distribute min_alloc equally among others
        others = [i for i in range(n_clusters) if i != idx]
        base = min_alloc // len(others)
        extra = min_alloc % len(others)
        for i, cluster in enumerate(others):
            cluster_sizes[cluster] += base + (1 if i < extra else 0)

    return cluster_sizes


In [79]:
def generate_synthetic_signed_graph(n_clusters=3, 
                                     total_nodes=20, 
                                     entropy=0.5,
                                     noise_level=0.0, 
                                     seed=None):
    if seed is not None:
        np.random.seed(seed)

    if n_clusters > total_nodes:
        raise ValueError("Number of clusters cannot exceed total number of nodes.")

    cluster_sizes = entropy_based_cluster_sizes(n_clusters=n_clusters, total_nodes=total_nodes, entropy=entropy, seed=seed)
    
    cluster_indices = []
    node_id = 0
    for size in cluster_sizes:
        indices = list(range(node_id, node_id + size))
        cluster_indices.append(indices)
        node_id += size

    adj_matrix = np.zeros((total_nodes, total_nodes))

    for indices in cluster_indices:
        for i in indices:
            for j in indices:
                if i != j:
                    value = np.random.uniform(0.1, 1.0)
                    adj_matrix[i, j] = value
                    adj_matrix[j, i] = value


    for i in range(total_nodes):
        for j in range(total_nodes):
            if adj_matrix[i, j] == 0 and i != j:
                value = np.random.uniform(-1.0, -0.1)
                adj_matrix[i, j] = value
                adj_matrix[j, i] = value

    noise = np.random.normal(0, noise_level, size=adj_matrix.shape)
    np.fill_diagonal(adj_matrix, 0)
    adj_matrix += noise
    adj_matrix = np.clip(adj_matrix, -1.0, 1.0)

    return adj_matrix, cluster_indices


## Clustering Metrics

In [80]:
def signed_modularity(adj_matrix, clusters):
    total_weight = np.sum(np.abs(adj_matrix)) / 2
    modularity = 0
    for cluster in clusters:
        for i, j in combinations(cluster, 2):
            modularity += adj_matrix[i, j]
    return modularity / total_weight

In [81]:
def penalty_metric(adj_matrix, clusters):
    penalty = 0
    for cluster in clusters:
        for i, j in combinations(cluster, 2):
            if adj_matrix[i, j] < 0:  # Penalize intra-cluster negative edges
                penalty += abs(adj_matrix[i, j])
    for i in range(len(adj_matrix)):
        for j in range(len(adj_matrix)):
            if i != j and adj_matrix[i, j] > 0:  # Penalize inter-cluster positive edges
                in_same_cluster = any(i in cluster and j in cluster for cluster in clusters)
                if not in_same_cluster:
                    penalty += adj_matrix[i, j]
    return penalty


## Running one problem instance

In [ ]:
n_clusters=5
total_nodes=170
seed = 111

adj_matrix, true_clusters = generate_synthetic_signed_graph(
    n_clusters=n_clusters,
    total_nodes=total_nodes,
    entropy=1, 
    noise_level=0.00,
    seed=seed
)

In [ ]:
true_clusters = sorted([sorted(cluster) for cluster in true_clusters])
print(true_clusters)
true_labels = clusters_to_labels(true_clusters, total_nodes)

In [72]:
# clusters_gcsq, total_qpu_access_time, total_partition_time = gcs_q_algorithm(adj_matrix, qubo_solver='gurobi')
# clusters_gcsq = sorted([sorted(cluster) for cluster in clusters_gcsq])
# labels_gcsq = clusters_to_labels(clusters_gcsq, total_nodes)
# print("clusters_gcsq",clusters_gcsq)
# print("gcsq",normalized_mutual_info_score(true_labels, labels_gcsq))

In [73]:
# t0 = time.time()
# clusters_gcsq, total_qpu_access_time, total_partition_time = gcs_q_algorithm(adj_matrix, qubo_solver='dwave')
# tte = time.time()-t0
# print("Remote", tte)
# print("Onsite",tte-total_partition_time+total_qpu_access_time)
# clusters_gcsq = sorted([sorted(cluster) for cluster in clusters_gcsq])
# labels_gcsq = clusters_to_labels(clusters_gcsq, total_nodes)
# print("clusters_gcsq",clusters_gcsq)
# print("gcsq",normalized_mutual_info_score(true_labels, labels_gcsq))

In [86]:
n_clusters=5
total_nodes=30
seed = 42

adj_matrix, true_clusters = generate_synthetic_signed_graph(
    n_clusters=n_clusters,
    total_nodes=total_nodes,
    entropy=0.5, 
    noise_level=0.00,
    seed=seed
)

true_clusters = sorted([sorted(cluster) for cluster in true_clusters])
print(true_clusters, penalty_metric(adj_matrix, true_clusters))
true_labels = clusters_to_labels(true_clusters, total_nodes)

clusters_gcsq, total_qpu_access_time, total_partition_time = gcs_q_algorithm(adj_matrix, qubo_solver='dwave')
clusters_gcsq = sorted([sorted(cluster) for cluster in clusters_gcsq])
labels_gcsq = clusters_to_labels(clusters_gcsq, total_nodes)
print("clusters_gcsq",clusters_gcsq)
print("gcsq",normalized_mutual_info_score(true_labels, labels_gcsq))

alpha = 2
distance_matrix = np.sqrt(alpha * (1 - adj_matrix.clip(min=-1, max=1)))
_, clusters_pam = pam(distance_matrix, k=n_clusters)
clusters_pam = sorted([sorted(cluster) for cluster in clusters_pam])
labels_pam = clusters_to_labels(clusters_pam, total_nodes)
print("\nclusters_pam",clusters_pam)
print("pam",normalized_mutual_info_score(true_labels, labels_pam))

clusters_kmeans = sorted([sorted(cluster) for cluster in run_kmeans(adj_matrix, k=n_clusters, seed=42)])
labels_kmeans = clusters_to_labels(clusters_kmeans, total_nodes)
print("\nclusters_kmeans",clusters_kmeans)
print("kmeans",normalized_mutual_info_score(true_labels, labels_kmeans))

clusters_hier = sorted([sorted(cluster) for cluster in run_hierarchical(adj_matrix, k=n_clusters)])
labels_hier = clusters_to_labels(clusters_hier, total_nodes)
print("\nclusters_hier",clusters_hier)
print("hier",normalized_mutual_info_score(true_labels, labels_hier))

clusters_spec = sorted([sorted(cluster) for cluster in run_spectral(adj_matrix, k=n_clusters, seed=42)])
labels_spec = clusters_to_labels(clusters_spec, total_nodes)
print("\nclusters_spec",clusters_spec)
print("spec",normalized_mutual_info_score(true_labels, labels_spec))

alpha = 2
distance_matrix = np.sqrt(alpha * (1 - adj_matrix.clip(min=-1, max=1)))
clusters_diana = sorted([sorted(cluster) for cluster in diana(distance_matrix, k=n_clusters)])
labels_diana = clusters_to_labels(clusters_diana, total_nodes)
print("\nclusters_diana",clusters_diana)
print("diana",normalized_mutual_info_score(true_labels, labels_diana))

[[0, 1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28, 29]] 0
clusters_gcsq [[0, 1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28, 29]]
gcsq 1.0

clusters_pam [[0, 1, 2, 3, 4, 5, 6], [7, 15, 16, 17, 20, 21, 22, 23, 24], [8, 26, 27, 28, 29], [9, 10, 11, 12], [13, 14, 18, 19, 25]]
pam 0.7511242502675732

clusters_kmeans [[0, 1, 2, 3, 4], [5, 6, 7, 8, 27], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 28, 29]]
kmeans 0.9458628120637362

clusters_hier [[0, 1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28, 29]]
hier 1.0

clusters_spec [[0, 1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28, 29]]
spec 1.0

clusters_diana [[0, 1, 2, 3, 4], [5, 29], [6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2

## Experiments and saving results

In [ ]:
def evaluate_and_log_clustering_results(solvers, metrics, adj_matrix, true_labels, total_nodes, n_clusters, results, qubo_solver = 'dwave'):
    def log_metrics(solver_name, clusters, duration):
        labels = clusters_to_labels(clusters, total_nodes)
        results[f"{solver_name}_clusters"] = clusters
        results[f"{solver_name}_labels"] = labels
        if solver_name == "GCSQ" and qubo_solver == "dwave":
            results[f"{solver_name}_Remote_Time"] = duration[0]
            results[f"{solver_name}_Onsite_Time"] = duration[1]
            print("Remote_Time:", round(duration[0],2), end=' ')
            print("Onsite_Time:", round(duration[1],2), end=' ')
        else:
            print(solver_name, round(duration,2), end=' ')
            results[f"{solver_name}_Time"] = duration

        if "NMI" in metrics:
            print(normalized_mutual_info_score(true_labels, labels))
            results[f"{solver_name}_NMI"] = normalized_mutual_info_score(true_labels, labels)
        if "Modularity" in metrics:
            results[f"{solver_name}_Modularity"] = signed_modularity(adj_matrix, clusters)
        if any(m in metrics for m in ["H", "C", "V"]):
            h, c, v = homogeneity_completeness_v_measure(true_labels, labels)
            if "H" in metrics: results[f"{solver_name}_H"] = h
            if "C" in metrics: results[f"{solver_name}_C"] = c
            if "V" in metrics: results[f"{solver_name}_V"] = v
        if "FMI" in metrics:
            results[f"{solver_name}_FMI"] = fowlkes_mallows_score(true_labels, labels)
        if "ARI" in metrics:
            results[f"{solver_name}_ARI"] = adjusted_rand_score(true_labels, labels)


    if "GCSQ" in solvers:
        t0 = time.time()
        clusters_gcsq, total_qpu_access_time, total_partition_time = gcs_q_algorithm(adj_matrix, qubo_solver=qubo_solver)
        tte = time.time() - t0
        if qubo_solver == 'dwave':
            remote_time = tte
            onsite_time = tte-total_partition_time+total_qpu_access_time
            duration = [remote_time,onsite_time]
        else:
            duration = tte
        clusters = sorted([sorted(cluster) for cluster in clusters_gcsq])
        log_metrics("GCSQ", clusters, duration)

    if "PAM" in solvers:
        alpha = 2
        distance_matrix = np.sqrt(alpha * (1 - adj_matrix.clip(min=-1, max=1)))
        t0 = time.time()
        _, clusters = pam(distance_matrix, k=n_clusters)
        duration = time.time() - t0
        clusters = sorted([sorted(cluster) for cluster in clusters])
        log_metrics("PAM", clusters, duration)

    if "KMeans" in solvers:
        t0 = time.time()
        clusters = sorted([sorted(cluster) for cluster in run_kmeans(adj_matrix, k=n_clusters, seed=42)])
        duration = time.time() - t0
        log_metrics("KMeans", clusters, duration)

    if "Hier" in solvers:
        t0 = time.time()
        clusters = sorted([sorted(cluster) for cluster in run_hierarchical(adj_matrix, k=n_clusters)])
        duration = time.time() - t0
        log_metrics("Hier", clusters, duration)

    if "Spectral" in solvers:
        t0 = time.time()
        clusters = sorted([sorted(cluster) for cluster in run_spectral(adj_matrix, k=n_clusters, seed=42)])
        duration = time.time() - t0
        log_metrics("Spectral", clusters, duration)

    if "DIANA" in solvers:
        t0 = time.time()
        alpha = 2
        distance_matrix = np.sqrt(alpha * (1 - adj_matrix.clip(min=-1, max=1)))
        clusters = sorted([sorted(cluster) for cluster in diana(distance_matrix, k=n_clusters)])
        duration = time.time() - t0
        log_metrics("DIANA", clusters, duration)

In [30]:

solvers = ["GCSQ", "PAM", "KMeans", "Hier", "Spectral"]
solvers = ["GCSQ"]

qubo_solver = "dwave"

metrics = ["clusters", "labels", "NMI", "Modularity", "H", "C", "V", "FMI", "ARI"]

fieldnames = ["seed", "total_nodes", "n_clusters", "entropy", "true_clusters", "true_labels"]


if "GCSQ" in solvers:
    fieldnames += [f"GCSQ_{metric}" for metric in metrics]
    if qubo_solver == "dwave":
        fieldnames += ["GCSQ_Remote_Time", "GCSQ_Onsite_Time"]
    else:
        fieldnames += ["GCSQ_Time"]

for solver in [s for s in solvers if s != "GCSQ"]:
    for metric in metrics + ["Time"]:
        fieldnames.append(f"{solver}_{metric}")

report_filename_csv = f"report_quantum.csv"

check_and_prepare_csv(report_filename_csv, fieldnames)

✅ File exists. Headers match. Will start appending.


In [ ]:
node_list = np.arange(10,171,10).tolist()
noise_level=0.00
seeds=[111,222,333,444,555]
entropy_list = [1.0, 0.75, 0.5, 0.25, 0.0]
entropy_list = [1.0, 0.5, 0.0]

for seed in seeds:
    for total_nodes in node_list:
        # print(f"Seed: {seed}, total_nodes: {total_nodes}")
        n_clusters_list = [5, 10, 15, 20]
        for n_clusters in n_clusters_list:
            if n_clusters > total_nodes:
                continue
            for entropy in entropy_list:
                print(f"Seed: {seed}, total_nodes: {total_nodes}, n_clusters: {n_clusters}")
                results = {}
                results["seed"] = seed
                results["total_nodes"] = total_nodes
                results["n_clusters"] = n_clusters
                results["entropy"] = entropy
    
                adj_matrix, true_clusters = generate_synthetic_signed_graph(n_clusters=n_clusters,
                                                                             total_nodes=total_nodes,
                                                                             entropy=entropy,
                                                                             noise_level=noise_level,
                                                                             seed=seed)
                true_labels = clusters_to_labels(true_clusters, total_nodes)
                results["true_clusters"] = true_clusters
                results["true_labels"] = true_labels
    
                evaluate_and_log_clustering_results(
                    solvers=solvers,
                    metrics=metrics,
                    adj_matrix=adj_matrix,
                    true_labels=true_labels,
                    total_nodes=total_nodes,
                    n_clusters=n_clusters,
                    results=results,
                    qubo_solver = qubo_solver
                )
    
                with open(report_filename_csv, mode='a', newline='') as report_file:
                    writer = csv.DictWriter(report_file, fieldnames=fieldnames)
                    row = {key: results.get(key, None) for key in fieldnames}
                    writer.writerow(row)